# Opioid Prescriber and Overdose Analysis

#### This notebook will explore the Opioid prescribers based on Medicare data from data.cms.gov. Then, it will use the Google Maps API to create a heat map of the Opioid prescriptions over the US.

Technologies:
* Google API
* Socrata
* Pandas

# Opioid Prescriber Investigation

In [1]:
# import dependencies
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
cms_client = Socrata("data.cms.gov", "dLbTEKbMpc1G4L7fCsOvHvATT")
#client = Socrata("data.cms.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cms.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

In [2]:
# First n results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
opioid_prescriber_results_2016 = cms_client.get("mm6p-bnrx", limit=2000000)

# Convert to pandas DataFrame
results_2016_df = pd.DataFrame.from_records(opioid_prescriber_results_2016)
results_2016_df.head()

,extended_release_opioid_claims,extended_release_opioid_prescribing_rate,npi,nppes_provider_first_name,nppes_provider_last_name,nppes_provider_state,nppes_provider_zip_code,opioid_claim_count,percent_opioid_claims,specialty_description,total_claim_count
0,NaN,NaN,1003000126,ARDALAN,ENKESHAFI,MD,21502,23,0.042201834862,Internal Medicine,545
1,63,0.062749003984,1003000142,RASHID,KHALIL,OH,43623,1004,0.57934218119,Anesthesiology,1733
2,0,0,1003000167,JULIO,ESCOBAR,NV,89403,11,0.22448979592,Dentist,49
3,0,NaN,1003000282,ROSIE,BLAKEMORE,TN,37243,NaN,NaN,Nurse Practitioner,146
4,0,0,1003000407,DAVID,GIRARDI,PA,15825,17,0.007640449438,Family Practice,2225


In [3]:
# sort the dataframe by top opioid prescriber (i.e. prescribers with
# highest 'opioid_claim_count')
sorted_results_2016 = results_2016_df.sort_values(by="opioid_claim_count", ascending=False)
sorted_results_2016.head()

,extended_release_opioid_claims,extended_release_opioid_prescribing_rate,npi,nppes_provider_first_name,nppes_provider_last_name,nppes_provider_state,nppes_provider_zip_code,opioid_claim_count,percent_opioid_claims,specialty_description,total_claim_count
672265,107,0.10710710711,1598729055,CHRISTOPHER,SMITH,PA,19610,999,0.064676939013,Internal Medicine,15446
547042,80,0.08008008008,1487639910,WILLIAM,GETSON,PA,16412,999,0.060158978682,Family Practice,16606
999453,22,0.022022022022,1881660942,JAMES,HARVEY,MS,38801,999,0.051084066271,Internal Medicine,19556
474256,NaN,NaN,1417963778,MAURICIO,BERMUDEZ,FL,33607,999,0.047968885048,Family Practice,20826
534227,24,0.024024024024,1477546976,CHARLES,HORSLEY,OH,45036,999,0.060970399756,Family Practice,16385


In [4]:
# create longitude and latitude columns to dump coordinates from Google API
# into in order to create heat map from
sorted_results_2016["lat"] = ""
sorted_results_2016["lng"] = ""

1131550


# CDC Information on Drug Overdoses

In [5]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
cdc_client = Socrata("data.cdc.gov", "dLbTEKbMpc1G4L7fCsOvHvATT")

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
cdc_results = cdc_client.get("a3uk-kgrx", limit=2000000)

# Convert to pandas DataFrame
cdc_results_df = pd.DataFrame.from_records(cdc_results)

In [7]:
# print list of indicators to explore
print(cdc_results_df["indicator"].unique())

array(['Number of Deaths', 'Number of Drug Overdose Deaths',
       'Percent with drugs specified', 'Heroin (T40.1)',
       'Opioids (T40.0-T40.4,T40.6)', 'Cocaine (T40.5)',
       'Natural & semi-synthetic opioids (T40.2)',
       'Synthetic opioids, excl. methadone (T40.4)',
       'Psychostimulants with abuse potential (T43.6)',
       'Methadone (T40.3)'], dtype=object)

In [13]:
# get data frame of opioids (it seems like the sub categories are included in these rows
# so only the 'Opioids (T40.0-T40.4,T40.6)' indicator was used)
# NOTE: Assumed 'data_value' column corresponded to the number of overdose deaths
opioids_cdc_df = cdc_results_df.loc[cdc_results_df["indicator"] == "Opioids (T40.0-T40.4,T40.6)", :]

opioids_cdc_df.head()

,data_value,footnote,indicator,month,percent_complete,percent_pending_investigation,predicted_value,state,state_name,year
7,684,NaN,"Opioids (T40.0-T40.4,T40.6)",May,100,0.00749105755004963,688,WA,WA,2015
21,413,NaN,"Opioids (T40.0-T40.4,T40.6)",September,100,0.118895966029724,424,NH,NH,2015
49,470,NaN,"Opioids (T40.0-T40.4,T40.6)",June,100,0.023597891921655,474,OK,OK,2015
50,447,Underreported due to incomplete data,"Opioids (T40.0-T40.4,T40.6)",March,99.5+,0.0103680663556247,448,OK,OK,2017
52,1198,NaN,"Opioids (T40.0-T40.4,T40.6)",March,100,0.306482096337539,1274,NY,NY,2015


In [24]:
# sense check data to get count of Opioid overdose deaths in 2016
opiod_overdose_deaths_2016 = pd.to_numeric(opioids_cdc_df.loc[opioids_cdc_df["year"] == "2016", :]['data_value']).sum()
print(f'There were {opiod_overdose_deaths_2016} opioid overdose deaths in the US in 2016.')

There were 602734 opioid overdose deaths in the US in 2016.


In [31]:
# create dataframe of 2016 Opioid overdose deaths by state
opioid_cdc_2016 = opioids_cdc_df.loc[opioids_cdc_df["year"] == "2016", ["data_value", "state"]]
opioid_cdc_2016 = opioid_cdc_2016.rename(columns={"data_value": "opioid_overdose_deaths"})
opioid_cdc_2016["opioid_overdose_deaths"] = pd.to_numeric(opioid_cdc_2016["opioid_overdose_deaths"])
# sum across the months to get year based values
opioid_cdc_2016 = opioid_cdc_2016.groupby(["state"]).sum()
opioid_cdc_2016.head()

,opioid_overdose_deaths
state,
CT,9528
DC,1989
MD,17918
ME,3222
NC,16343


In [32]:
# sort the dataframe by most deaths
opioid_cdc_2016_sorted = opioid_cdc_2016.sort_values(by="opioid_overdose_deaths", ascending=False)
opioid_cdc_2016_sorted.head()

,opioid_overdose_deaths
state,
US,452369
NY,20733
MD,17918
NC,16343
VA,11715


In [36]:
opioid_check = opioid_cdc_2016_sorted.loc[opioid_cdc_2016_sorted.index != "US", :]
print(opioid_check["opioid_overdose_deaths"].sum())

150365
